In [2]:
import numpy as np
import pandas as pd

In [15]:
players_2324_path = './Quotazioni_Fantacalcio_Stagione_2023_24.csv'
players_2223_path = './Quotazioni_Fantacalcio_Stagione_2022_23.csv'
stats_2223_path = './Statistiche_Fantacalcio_Stagione_2022_23.csv'

players_2324_df = pd.read_csv(players_2324_path, delimiter=',', header=1, index_col=False)
players_2223_df = pd.read_csv(players_2223_path, delimiter=',', header=1, index_col=False)
stats_2223_df = pd.read_csv(stats_2223_path, delimiter=',', header=1, index_col=False)

In [8]:
players_2324_df[players_2324_df['FVM M'] > 30]

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
0,572,P,Por,Meret,Napoli,17,17,0,17,17,0,80,80
1,4312,P,Por,Maignan,Milan,17,17,0,17,17,0,89,89
2,2814,P,Por,Provedel,Lazio,16,16,0,16,16,0,68,68
3,453,P,Por,Szczesny,Juventus,15,15,0,15,15,0,71,71
4,2428,P,Por,Sommer,Inter,15,15,0,15,15,0,67,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,1958,A,A,Caprari,Monza,14,14,0,14,14,0,54,60
467,2038,A,Pc,Pinamonti,Sassuolo,13,13,0,12,12,0,48,40
469,5311,A,Pc,Shomurodov,Cagliari,13,13,0,13,13,0,40,34
475,6145,A,A,Ngonge,Verona,12,12,0,12,12,0,47,52


In [17]:
def extract_players_mantra_position(df: pd.DataFrame, position: str) -> pd.DataFrame:
    return df[df['RM'].str.contains(position)]


In [18]:
extract_players_mantra_position(players_2324_df, 'Dc')

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
65,6066,D,Dd;Dc,Posch,Bologna,16,16,0,18,18,0,53,56
67,4237,D,Dd;Ds;Dc,Danilo,Juventus,15,15,0,18,18,0,44,58
68,4245,D,Dc,Smalling,Roma,15,15,0,15,15,0,55,55
69,4409,D,Dc,Rrahmani,Napoli,14,14,0,14,14,0,44,44
70,6041,D,Dc,Schuurs,Torino,14,14,0,14,14,0,46,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,6244,D,Dc,Matturro,Genoa,1,1,0,1,1,0,1,1
239,6253,D,Dc,N'guessan,Torino,1,1,0,1,1,0,1,1
243,6411,D,Dc,Mateus Lusuardi,Frosinone,1,1,0,1,1,0,1,1
244,6430,D,Dc,Kalaj,Frosinone,1,1,0,1,1,0,1,1


In [11]:
fantacalcio_df[150:160]['RM']

150      Ds;Dc
151       Ds;E
152       Ds;E
153         Dc
154         Dc
155         Dc
156    Dd;Ds;E
157         Dc
158         Dc
159         Dc
Name: RM, dtype: object